In [ ]:
# Import libraries

import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random

In [ ]:
# json that describe the database

JSON = {
  "title.akas.tsv.gz": [
    {
      "titleId": "a tconst, an alphanumeric unique identifier of the title",
      "ordering": "a number to uniquely identify rows for a given titleId",
      "title": "the localized title",
      "region": "the region for this version of the title",
      "language": "the language of the title",
      "types": [
        "alternative",
        "dvd",
        "festival",
        "tv",
        "video",
        "working",
        "original",
        "imdbDisplay"
      ],
      "attributes": "Additional terms to describe this alternative title, not enumerated",
      "isOriginalTitle": "0: not original title; 1: original title"
    }
  ],
  "title.basics.tsv.gz": [
    {
      "tconst": "alphanumeric unique identifier of the title",
      "titleType": "the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)",
      "primaryTitle": "the more popular title / the title used by the filmmakers on promotional materials at the point of release",
      "originalTitle": "original title, in the original language",
      "isAdult": "0: non-adult title; 1: adult title",
      "startYear": "represents the release year of a title. In the case of TV Series, it is the series start year",
      "endYear": "TV Series end year. ‘\\N’ for all other title types",
      "runtimeMinutes": "primary runtime of the title, in minutes",
      "genres": "up to three genres associated with the title"
    }
  ],
  # and so on for the other files
}

In [ ]:
# Define dataset

# Possible queries
# I inflated the database using chatgpt
temp = """How many movies were released in the last month?
How many actors were born in August?
What is the language of the movie "Pulp Fiction"?
What is the series with the most episodes?
In how many horror movies did Sean Connery
What is the average rating of the movies directed by Quentin Tarantino?
What is the name of the movie with the highest rating?
Who is the director of the movie “The Godfather”?
What is the genre of the movie “The Shawshank Redemption”?
What is the name of the actor who played James Bond in “Goldfinger”?
How many movies were released in 2022?
What is the name of the movie with the most nominations for an Oscar?
Who is the actress with the most nominations for an Oscar?
What is the name of the movie with the longest runtime?
How many movies were released in 2021 and have a rating above 8?
What is the name of the actor who played Batman in “The Dark Knight”?
Who is the director of “The Lord of The Rings” trilogy?
What is the name of the movie that won Best Picture in 2020?
How many movies were released in 2020 and have a runtime above 3 hours?
Who is the actress who played Hermione Granger in “Harry Potter and The Philosopher’s Stone”?"""
df = pd.DataFrame(temp.split('\n'),columns = ['questions'])

df.head(3)


,questions
0,How many movies were released in the last month?
1,How many actors were born in August?
2,"What is the language of the movie ""Pulp Fiction""?"


In [ ]:
def OAI_API(text:str,prompt:str) -> str:
  # this function using API of OpenAI an generate text
  # for this home test I genarte the same text as the question, but in apper case
  return text.upper()


In [ ]:
 # break the question and inflated. TODO: the prompts need to fine tune

 TEXT_INFLAT_QUESTION = " Another way to ask this is: "
 PROMPT_BREAK_TO_TASK = "Rewrite the question. Keep only relevant information in your output. Organize the text and break it into separate tasks"
 df['questions_inflated'] = (
    df['questions'] +
    TEXT_INFLAT_QUESTION +
    df['questions'].apply(OAI_API,args = (PROMPT_BREAK_TO_TASK,))
 )
 df.head(2)

,questions,questions_inflated
0,How many movies were released in the last month?,How many movies were released in the last mont...
1,How many actors were born in August?,How many actors were born in August? Another w...


In [ ]:
# Searched for the best profmat

prompt_1 = "The database is described at the JSON [JSON]. [Concatenate question] is my question. How sure are you that you know the answer? Rate your confidence from 1 to 5, where 1 is very unsure and 5 is very sure"
prompt_2 = "I have given a description of the database at the JSON [JSON]. My question is this: [concatenate question]. How confident are you in your ability to answer it? Give a rating from 1 to 5, with 1 being the lowest and 5 being the highest level of confidence"
prompt_3 = "At the JSON, you can find a description of the database [JSON]. This is my question: [concatenate question]. How do you rate your confidence in answering it? Use a scale from 1 to 5, where 1 means not confident at all and 5 means very confident"
prompt_4 = "The database is explained at the JSON [JSON]. I want to ask you this: [concatenate question]. How certain are you that you have the answer? Choose a number from 1 to 5, where 1 indicates very uncertain and 5 indicates very certain"
prompt_5 = "You can see a description of the database at the JSON [JSON]. This is what I want to know: [concatenate question]. How do you feel about your confidence in answering it? Pick a number from 1 to 5, where 1 represents very low confidence and 5 represents very high confidence"

prompts = [prompt_1,prompt_2,prompt_3,prompt_4,prompt_5]


def OAI_API(text:str):
  return random.randint(1,len(prompts))

def OAI_API_PROMPT_ENGENERE(row):
  row['number prompt selected'] = random.randint(0,len(prompts) - 1)
  prompt = prompts[row['number prompt selected']]
  prompt = prompt.replace('[JSON]',str(JSON)).replace('[concatenate question]',row['questions_inflated'])
  row['confident'] = OAI_API(prompt)
  return row
df = df.apply(OAI_API_PROMPT_ENGENERE,axis = 1)
df.sample(4)

,questions,questions_inflated,number prompt selected,confident
0,How many movies were released in the last month?,How many movies were released in the last mont...,4,2
8,What is the genre of the movie “The Shawshank ...,What is the genre of the movie “The Shawshank ...,2,5
15,What is the name of the actor who played Batma...,What is the name of the actor who played Batma...,1,5
2,"What is the language of the movie ""Pulp Fiction""?","What is the language of the movie ""Pulp Fictio...",2,4


In [ ]:
# Visualization of the results

fig = go.Figure(
    data=[go.Bar(y=df.groupby('number prompt selected')['confident'].mean())],
    layout_title_text="A Figure Displayed with fig.show()"
)
fig.show()

In [ ]:
"""
It seems that prompt 0 and 3 are good candidates.
The next step is to implement a/b testing logic.
As a master’s degree holder in statistics, this is a relatively easy part for me but due to lack of time I did not implement it.

"""